In [1]:
library(pogit)
# PIP, , what exactly do these 2 weights mean
# how to deal with the intercept: did not make difference if centering

In [3]:
block = c(79,137)
block_simu = c(1215, 1273)
# block = c(23, 36)
# block_simu = c(164, 177)

In [4]:
X_ori <- as.matrix(data.table::fread(paste("/data/GIT/cnv-gene-mapping/data/deletion_simu/block_", block[1], "_" , block[2], 
                                           "/deletion.genes.block30.for_simu.sample.genes.block_", block[1], "_" , block[2], ".gz", sep = ""), header = F))

In [5]:
X_center <- scale(X_ori, scale = FALSE)

In [6]:
head(X_center)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05


In [7]:
dim(X_ori)

[1] 13412    59

In [8]:
y <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.y"))

In [9]:
length(y)

[1] 13412

In [9]:
# beta <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.beta"))

In [10]:
# mean(beta); sd(beta); sum(beta == 0); sum(beta == 0)/length(beta)

In [29]:
X_intercept <- cbind(rep(1, nrow(X_ori)), X_ori)

In [10]:
y <- as.numeric(y[,1])

In [11]:
N <- rep(1, length(y))

In [12]:
prior <- readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.binomial.whole.genome.prior.varbvs.nologodds.updateb0.rds")

In [45]:
mu0 = sum(prior$b0 * prior$w)

In [46]:
mu0

[1] 0.7770721

In [15]:
var = sum(prior$sa * prior$w)

In [16]:
var

[1] 0.7117456

In [17]:
pi0 = sum((10^(prior$logodds) / (1 + 10^(prior$logodds))) * prior$w)

In [18]:
pi0

[1] 0.0437755

In [21]:
# b = prior$beta[which(prior$beta != 0)]
# mean(b); var(b); sum(prior$beta == 0); sum(prior$beta == 0)/length(prior$beta)

In [19]:
X <- X_center

In [ ]:
res <- logitBvs(y, N, X, prior = list(slab = "Normal", aj0 = rep(mu0, ncol(X_ori)), V = var, w = c(wa0 = 1, wb0 = 1), pi = c(pa0 = 1.412, pb0 = 10)), 
                mcmc = list(M = 5000, burnin = 1000, msave = TRUE), BVS = FALSE)
# res1 <- logitBvs(y, N, X, prior = list(slab = "Normal", aj0 = rep(mu0, ncol(X_ori)), V = var, w = c(wa0 = 1.412, wb0 = 10), pi = c(pa0 = 1, pb0 = 1)), 
#                  mcmc = list(M = 5000, burnin = 1000, msave = TRUE), BVS = TRUE)


MCMC for the binomial logit model:

it = 1 /--- duration of MCMC so far: 25.1 sec.,  expected time to end: Inf  min. 
it = 2 /--- duration of MCMC so far: 25.23 sec.,  expected time to end: 2522.18  min. 
it = 3 /--- duration of MCMC so far: 25.37 sec.,  expected time to end: 1268.13  min. 
it = 4 /--- duration of MCMC so far: 25.5 sec.,  expected time to end: 849.58  min. 
it = 5 /--- duration of MCMC so far: 25.66 sec.,  expected time to end: 641.05  min. 
it = 10 /--- duration of MCMC so far: 26.26 sec.,  expected time to end: 291.33  min. 
it = 20 /--- duration of MCMC so far: 27.68 sec.,  expected time to end: 145.21  min. 
it = 50 /--- duration of MCMC so far: 31.47 sec.,  expected time to end: 63.7  min. 
it = 100 /--- duration of MCMC so far: 37.39 sec.,  expected time to end: 37.15  min. 
it = 200 /--- duration of MCMC so far: 48.93 sec.,  expected time to end: 23.77  min. 
it = 500 /--- duration of MCMC so far: 83.96 sec.,  expected time to end: 15.43  min. 


In [48]:
res_pip <- logitBvs(y, N, X, prior = list(slab = "Normal", aj0 = rep(0, ncol(X_ori)), V = var, w = c(wa0 = 1.412, wb0 = 10), pi = c(pa0 = 1, pb0 = 1)), 
                    mcmc = list(M = 5, burnin = 1, msave = TRUE), BVS = TRUE)

Warning message:
“invalid 'BVS' or 'startsel' argument:
'startsel' was set to 0.5”


MCMC for the binomial logit model with variable selection:

it = 1 /--- duration of MCMC so far: 25.22 sec.,  expected time to end: Inf  min. 
it = 2 /--- duration of MCMC so far: 31.43 sec.,  expected time to end: 2.62  min. 
it = 3 /--- duration of MCMC so far: 36.56 sec.,  expected time to end: 1.22  min. 
it = 4 /--- duration of MCMC so far: 41.13 sec.,  expected time to end: 0.69  min. 
it = 5 /--- duration of MCMC so far: 45.67 sec.,  expected time to end: 0.38  min. 
it = 6 /--- duration of MCMC (total): 50.33 sec. 
 


In [ ]:
head(summary(res_pip))

In [ ]:
summary(res)

In [50]:
summary(res1)

MCMC for the logit model:

Call:
logitBvs(y = y, N = N, X = X, prior = list(slab = "Normal", aj0 = rep(mu0, 
    ncol(X_ori)), V = var, w = c(wa0 = 1.412, wb0 = 10), pi = c(pa0 = 1, 
    pb0 = 1)), mcmc = list(M = 5000, burnin = 1000, msave = TRUE), 
    BVS = TRUE)


MCMC:
M = 5000 draws after a burn-in of 1000
Thinning parameter: 1

Prior: Normal prior [V=0.711745609189383]

 a0[0]  a0[1]  a0[2]  a0[3]  a0[4]  a0[5]  a0[6]  a0[7]  a0[8]  a0[9] a0[10] 
 0.000  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[11] a0[12] a0[13] a0[14] a0[15] a0[16] a0[17] a0[18] a0[19] a0[20] a0[21] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[22] a0[23] a0[24] a0[25] a0[26] a0[27] a0[28] a0[29] a0[30] a0[31] a0[32] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[33] a0[34] a0[35] a0[36] a0[37] a0[38] a0[39] a0[40] a0[41] a0[42] a0[43] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.7

In [21]:
# saveRDS(res, paste("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/logitBvs.block_", block[1], "_", block[2],".var.selection.res.rds", sep = ""))

In [1]:
# res <- readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/logitBvs.var.selection.res.rds")

In [51]:
tmp <- res1

In [52]:
names(tmp)

[1] "samplesL"    "data"        "model.logit" "mcmc"        "prior.logit"
 [6] "dur"         "BVS"         "start"       "family"      "call"

In [53]:
names(tmp$samplesL)

[1] "alpha"       "pdeltaAlpha" "psiAlpha"    "thetaAlpha"  "pgammaAlpha"
[6] "ai"

In [54]:
head(tmp$samplesL$alpha)

alpha.0,alpha.1,alpha.2,alpha.3,alpha.4,alpha.5,alpha.6,alpha.7,alpha.8,alpha.9,⋯,alpha.50,alpha.51,alpha.52,alpha.53,alpha.54,alpha.55,alpha.56,alpha.57,alpha.58,alpha.59
-0.008922104,-0.53034624,0.531186533,0.4457819,0.1599986,-1.3182861,0.09927597,0.040819133,0.14441332,-1.20171809,⋯,-0.4265390,0.3910205,0.46967763,0.5613216,1.324210991,-1.5534997,-1.6478087,-1.075354152,0.1485861,1.2174901
0.008121581,0.79518523,0.954229860,-0.1130958,-0.7076055,1.0144555,0.06313414,0.500058544,-0.05249471,0.02847695,⋯,-0.5672389,-0.2361822,0.69763880,-0.9993369,-0.003138283,0.1564388,-0.2261067,0.178268803,0.8484847,1.5755036
0.004549283,-0.49674451,-0.988052445,-0.5637793,1.0699240,1.3984557,-0.06392324,-0.002354454,-0.54175689,-0.58197656,⋯,1.4600980,-0.9431929,-0.52161259,0.3376865,-0.529398446,0.5669247,0.5654980,0.006672887,0.2599798,-0.6937189
-0.015049224,-0.13860589,0.293100422,-0.1334541,1.2996531,1.4361739,-1.19321224,0.508942331,0.26084063,0.32264172,⋯,0.2647752,-1.3852779,0.78115515,0.1477897,-0.534497911,-0.8138973,-1.4152443,1.090435096,-0.7521541,0.9998119
-0.035771400,0.06670708,0.009164063,-0.7054603,0.3679066,-1.3005890,-0.02348997,0.197857705,1.90309204,-0.90931598,⋯,-0.2912868,0.6487994,-0.07720553,-0.3198757,-0.346299501,0.2265415,-0.2786059,0.926508434,-0.5175433,-1.0035232
-0.015940786,-1.33713301,1.018747459,-1.2060913,-0.7895486,-0.3490836,0.63312591,1.465485144,-0.17208142,0.33057663,⋯,-0.2077031,0.5747511,-0.31270666,0.4434742,-1.602214803,1.0651499,-1.0111447,0.510094357,-0.8205276,-0.5999659


In [55]:
bhat <- colMeans(tmp$samplesL$alpha[,-1])
bhat1 <- colMeans(tmp$samplesL$alpha)

In [56]:
# yhat <- exp(X %*% bhat)/(1+exp(X %*% bhat))
yhat <- X_ori %*% bhat
yhat1 <- X_center %*% bhat
yhat2 <- X_intercept %*% bhat1

In [57]:
min(yhat); max(yhat); mean(yhat); median(yhat)

[1] -1.079004

[1] 1.650473

[1] 0.001295595

[1] 0

In [58]:
b_all <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.beta"))
b <- b_all[(block_simu[1]+1):(block_simu[2]+1)]

In [59]:
# y <- exp(X %*% b)/(1+exp(X %*% b))
yb <- X_ori %*% b

In [60]:
min(yb); max(yb); mean(yb); median(yb)

[1] 0

[1] 0.9578892

[1] 0.004438093

[1] 0

In [63]:
cor(yb, yhat, method = "pearson")

0.3444027


In [39]:
min(yhat1[yhat1<0])

[1] -0.111294

In [40]:
cor.test(yb, yhat, method=c("pearson"))


	Pearson's product-moment correlation

data:  yb and yhat
t = 874.81, df = 13410, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.9910558 0.9916388
sample estimates:
      cor 
0.9913522 


In [26]:
summary(tmp)

Bayesian variable selection for the logit model:

Call:
logitBvs(y = y, N = N, X = X)


MCMC:
M = 8000 draws after a burn-in of 2000
BVS started after 1000 iterations
Thinning parameter: 1

Prior:  spike-and-slab prior with Student-t slab [V=5]

 a0[0]  a0[1]  a0[2]  a0[3]  a0[4]  a0[5]  a0[6]  a0[7]  a0[8]  a0[9] a0[10] 
     0      0      0      0      0      0      0      0      0      0      0 
a0[11] a0[12] a0[13] a0[14] 
     0      0      0      0 
 w[a]  w[b] pi[a] pi[b] 
    1     1     1     1 


Model averaged posterior means, estimated posterior inclusion
 probabilities and 95%-HPD intervals:

            Estimate P(.=1) 95%-HPD[l] 95%-HPD[u]
(Intercept)    0.029     NA     -0.012      0.067
alpha.1        0.216  0.512     -0.099      0.916
alpha.2        0.426  0.543     -2.258      3.523
alpha.3        0.458  0.541     -2.258      3.663
alpha.4        0.448  0.541     -2.249      3.660
alpha.5        0.446  0.542     -2.097      3.782
alpha.6        0.449  0.544     -2.23

In [92]:
pdf(file = paste("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/logitBvs.block_", block[1], "_", block[2],".var.selection.beta.hist.pdf", sep = ""))
par(mfrow = c(2,2))
for (i in seq(2, (block[2]-block[1]+2))){
    hist(res$samplesL$alpha[,i], breaks = 80, main = paste("beta", i-1, "histogram"), xlab = paste("beta", i-1))
}
dev.off()

png 
  2

In [94]:
dim(res$samplesL$alpha)

[1] 12000    15

In [79]:
# mean(res$samplesL$alpha[,42])

In [78]:
summary(res)

MCMC for the logit model:

Call:
logitBvs(y = y, N = N, X = X, prior = list(slab = "Normal", aj0 = rep(mu0, 
    ncol(X) - 1), V = var, w = c(wa0 = 1, wb0 = 1), pi = c(pa0 = 1.412, 
    pb0 = 10)), mcmc = list(M = 24000, burnin = 6000, msave = TRUE), 
    BVS = FALSE)


MCMC:
M = 24000 draws after a burn-in of 6000
Thinning parameter: 1

Prior: Normal prior [V=0.711745609189383]

 a0[0]  a0[1]  a0[2]  a0[3]  a0[4]  a0[5]  a0[6]  a0[7]  a0[8]  a0[9] a0[10] 
 0.000  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[11] a0[12] a0[13] a0[14] a0[15] a0[16] a0[17] a0[18] a0[19] a0[20] a0[21] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[22] a0[23] a0[24] a0[25] a0[26] a0[27] a0[28] a0[29] a0[30] a0[31] a0[32] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[33] a0[34] a0[35] a0[36] a0[37] a0[38] a0[39] a0[40] a0[41] a0[42] a0[43] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  

In [70]:
data(simul_binomial)

In [71]:
y <- simul_binomial$y
N <- simul_binomial$N
X <- as.matrix(simul_binomial[, -c(1,2)])

In [72]:
length(y); length(N); dim(X)

[1] 512

[1] 512

[1] 512  10

In [73]:
head(X)

X.0,X.1,X.2,X.3,X.4,X.5,X.6,X.7,X.8,X.9
1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,0,1


In [74]:
prior = list(slab = "Normal", aj0 = rep(mu0, ncol(X_ori)), V = var, w = c(wa0 = 1, wb0 = 1), pi = c(pa0 = 1.412, pb0 = 10))

[1] 512

In [87]:
m3 <- logitBvs(y = y, N = N, X = X, mcmc = list(M = 4, burnin = 1), prior = list(slab = "Normal", aj0 = rep(0, ncol(X)-1)), 
               model = list(deltafix = c(1, 1, 1, 0, 0, 0, 1, 0, 0)))

Warning message:
“invalid 'BVS' or 'startsel' argument:
'startsel' was set to 0.5”


MCMC for the binomial logit model with variable selection:

it = 1 /--- duration of MCMC so far: 1.1 sec.,  expected time to end: Inf  min. 
it = 2 /--- duration of MCMC so far: 1.11 sec.,  expected time to end: 0.07  min. 
it = 3 /--- duration of MCMC so far: 1.13 sec.,  expected time to end: 0.03  min. 
it = 4 /--- duration of MCMC so far: 1.15 sec.,  expected time to end: 0.01  min. 
it = 5 /--- duration of MCMC so far: 1.17 sec.,  expected time to end: 0  min. 


In [88]:
summary(m3)

Bayesian variable selection for the binomial logit model:

Call:
logitBvs(y = y, N = N, X = X, model = list(deltafix = c(1, 1, 
    1, 0, 0, 0, 1, 0, 0)), prior = list(slab = "Normal", aj0 = rep(0, 
    ncol(X) - 1)), mcmc = list(M = 4, burnin = 1))


MCMC:
M = 4 draws after a burn-in of 1
BVS started after 0.5 iterations
Thinning parameter: 1

Prior:  spike-and-slab prior with Normal slab [V=5]

a0[0] a0[1] a0[2] a0[3] a0[4] a0[5] a0[6] a0[7] a0[8] a0[9] 
    0     0     0     0     0     0     0     0     0     0 
 w[a]  w[b] pi[a] pi[b] 
    1     1     1     1 


Model averaged posterior means, estimated posterior inclusion
 probabilities and 95%-HPD intervals:

            Estimate P(.=1) 95%-HPD[l] 95%-HPD[u]
(Intercept)   -0.419     NA     -0.483     -0.363
alpha.1        0.156  1.000      0.130      0.190
alpha.2       -0.106  1.000     -0.124     -0.075
alpha.3        0.072  1.000      0.019      0.127
alpha.4       -0.874  1.000     -0.946     -0.726
alpha.5        0.000  0.0